Gleened from https://medium.com/@galen.ballew/transferlearning-b65772083b47



In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing import image

from keras.models import Sequential
import numpy

#import inception with pre-trained weights. do not include fully #connected layers
VGG16_base = VGG16(weights='imagenet', include_top=False)

In [ ]:
x = VGG16_base.output
x = GlobalAveragePooling2D()(x)

# create the full network so we can train on it
VGG16_Bottle = Model(input=VGG16_base.input, output=x)


In [ ]:
# This part has been modified to download and image into a temp folder
# Resize temp image, feed it throught the network
# Based on the hazard rating give the appropriate label

import PIL 
import os
from PIL import Image
# Generate dummy data
import numpy as np
import random
import photoSort as ps
import pandas as pd
#data = np.random.random((1000, 155,200,3))
#labels = np.random.randint(2, size=(1000, numberOfClasses))

#VGG16_transfer.fit(data,labels, epochs=4, batch_size=16)
    
## !! SET THESE PARAMETERS !! ##
NumIm = 500 # The number of images you want
maxInt = 7172 # The max entry of the csv file
Its = 0
numD = np.round(NumIm/2)  # Number of daisies


data = np.zeros([NumIm,263,320,3])  # Initialize an array for all the data
outData= np.zeros([NumIm,512])
outLabels=np.zeros([NumIm,2])

# Import the csv file
directory = 'CSVliveshere/AsFoundPhotoIndex-Table1.csv'
df = pd.read_csv(directory)

# Now we will loop through the samples
samples = random.sample(range(maxInt+1), NumIm) # This generates a random sample

totalSize = 0
low_number =0
high_number=0

# Loop through the samples and download the images into the correct folders (high or low)
for k in samples:
    photoLink = df['PHOTOLINK'][k]
    hazardRating = df['HAZARDMAPRATING'][k]
    
    category,size = ps.download_image_into_tmp(photoLink,hazardRating )
    
    img = image.load_img('./tmp/temp.png', target_size=(224, 224))
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = VGG16_Bottle.predict(x)
    outData[Its,:]=preds[0,:]
    
    if(category=='low'):
        outLabels[Its,0]=1
        low_number += 1
    else:
        outLabels[Its,1]=1
        high_number += 1
    Its += 1
    totalSize += size

print('\n High Number: '+ str(high_number)+' '+ str(float(high_number)/float(NumIm)))
print('\n Low Number: '+ str(low_number)+ ' '+ str(float(low_number)/float(NumIm)))
print('\n Total Download Size: '+str(totalSize)+' Mbs')
    # Now that image has been placed into tmp, load it, 

# Need to pull an image from the cloud
# And then based on the Hazard rating, give the appropriate label
# Low  = [1,0]
# High = [0,1]

In [ ]:
outData=outData[1:Its-1,:]
outLabels=outLabels[1:Its-1,:]

In [ ]:

np.savetxt("BCSAWeights.csv",outData)
np.savetxt("BCSALabels.csv",outLabels)

In [ ]:
outData= np.loadtxt("BCSAWeights.csv")
outLabels= np.loadtxt("BCSALabels.csv")





In [ ]:
myPerm= numpy.random.permutation(outData.shape[0])
trainSize= round(0.8*outData.shape[0])

In [ ]:
outData=outData[myPerm,:]
outLabels=outLabels[myPerm,:]

trainData= outData[:trainSize,:]
trainLabel= outLabels[:trainSize,:]
testData= outData[trainSize:,:]
testLabel= outLabels[trainSize:,:]

In [ ]:
testLabel.shape

In [ ]:
newModel = Sequential()
newModel.add(Dense(256, input_dim=512, activation='relu'))
newModel.add(Dense(256, activation='relu'))
newModel.add(Dense(2, activation='sigmoid'))

newModel.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
newModel.fit(trainData,trainLabel, nb_epoch=150, batch_size=128)

In [ ]:
newModel.evaluate(testData,testLabel)[1]*100

Getting 99% accuracy on the flower data.
Also fast.